In [4]:
!pip install datasets
!pip install transformers
!pip install vertexai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.7/333.7 kB 17.2 MB/s eta 0:00:00


In [3]:
# Pretty print
from pprint import pprint
# Datasets load_dataset function
from datasets import load_dataset
# Transformers Autokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# Standard PyTorch DataLoader
from torch.utils.data import DataLoader

from PIL import Image as PILImage

In [6]:
def concat_images(images):
    if len(images) == 1:
        return images[0]
    if len(images) < 1:
        return None
    max_height = 0
    total_width = 0
    for image in images:
        width, height = image.size
        print(width, height)
        max_height = max(height, max_height)
        total_width += width
    to_return = PILImage.new("RGB", (total_width, max_height))
    curr_x = 0
    for image in images:
        to_return.paste(image, (curr_x, 0))
        curr_x += image.size[0]
    # to_return.save("out.png")
    return to_return

In [4]:
dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

/home/ubuntu/cs477-final-project/.conda/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for HUPD/hupd contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HUPD/hupd
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset with config: PatentsConfig(name='sample', version=0.0.0, data_dir='sample', data_files={'train': ['https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather']}, description='Patent data from January 2016, for debugging')


Using metadata file: /home/ubuntu/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710


Reading metadata file: /home/ubuntu/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710
Filtering train dataset by filing start date: 2016-01-01
Filtering train dataset by filing end date: 2016-01-21
Filtering val dataset by filing start date: 2016-01-22
Filtering val dataset by filing end date: 2016-01-31


Generating train split: 16153 examples [00:19, 821.42 examples/s] 
Generating validation split: 9094 examples [00:10, 855.30 examples/s] 

Loading is done!


In [17]:
# Azure prompts
def generate_prompt_openai(patent_dict):
    """
    Returns:
        messages: list to use as input to client.chat.completions.create()
        target: groundtruth patent class label
    """
    messages = [
        { "role": "user", "content": "Classify the following patent into a class in the International Patent Classification system. Answer with the a 4-character IPC class."},
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": patent_dict["abstract"]
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "TODO"
                } # you can also upload as a base64 images lol
            }
        ] } 
    ]
    return messages, patent_dict["ipc_label"][:4]

In [8]:
print(generate_prompt_openai(dataset_dict['train'][0]))

([{'role': 'user', 'content': 'Classify the following patent into a class in the International Patent Classification system. Answer with the a 4-character IPC class.'}, {'role': 'user', 'content': [{'type': 'text', 'text': 'The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'}, {'type': 'image_url', 'image_url': {'url': 'TODO'}}]}], 'H04Q')


In [18]:
# Azure prompts
from openai import AzureOpenAI
api_base = 'https://gerstein-westus.openai.azure.com/'
api_key="b4521b5444d74d0284e9aa797245b25d"
deployment_name = 'gpt4-vision-preview'
api_version = '2023-12-01-preview'

client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}/extensions",
)

messages, ground_truth = generate_prompt_openai(dataset_dict['train'][0])
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=2000 
)
print(response)

ChatCompletion(id='chatcmpl-9FBYB6cq76llMSeFkiBokOb1gMm1r', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='G06K 19/00', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1713406347, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=227, total_tokens=234), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [ ]:
# LLaVA prompts

def generate_prompt_llava(patent_dict):
    messages = [
        {   "id": hash(str(patent_dict["patent_number"])),
            "image": "TODO",
            "conversations": [
                {
                    "from": "human",
                    "value": "<image>\nClassify the following patent into a class in the International Patent Classification system. Answer with the a 4-character IPC class.\n" + patent_dict["abstract"]
                },
                {
                    "from": "gpt",
                    "value": "CLASSIFICATION"
                }
            ]
        }
    ]
    return messages, patent_dict["ipc_label"][:4]

In [20]:
from vertexai.generative_models import Image as VertexImage
import urllib.request
import http.client
import typing


# Gemini prompts

# based on input format of https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts

def load_image_from_url(image_url):
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return VertexImage.from_bytes(image_bytes)

def generate_prompt_gemini(patent_dict):
    messages = [
        load_image_from_url("https://patentimages.storage.googleapis.com/23/82/97/d28e8e17f1597a/US20140002137A1-20140102-D00002.png"),
        "Classify the following patent into a class in the International Patent Classification system. Answer with the a 4-character IPC class.\n" + patent_dict["abstract"]
    ]
    return messages, patent_dict["ipc_label"][:4]

In [21]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

vertexai.init(project="cpsc477-patentpublic", location="us-central1")

model = GenerativeModel("gemini-1.0-pro-vision")

messages, ground_truth = generate_prompt_gemini(dataset_dict['train'][0])
print(messages)

response = model.generate_content(messages)

print(response)

[<vertexai.generative_models._generative_models.Image object at 0x7f8db4195250>, 'Classify the following patent into a class in the International Patent Classification system. Answer with the a 4-character IPC class.\nThe present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.']
candidates {
  finish_reason: OTHER
}
usage_metadata {
  prompt_token_count: 379
  total_token_count: 379
}



In [22]:
response.text

AttributeError: Content has no parts.